# 📐 2D Lattice Basis Reduction – Interactive Notebook

This notebook demonstrates the step-by-step process of **reducing a two-dimensional lattice basis** using a simple Gram-Schmidt–style algorithm.

## 🔍 What it does:
- Takes an initial integer basis `{b1, b2}` of a 2D lattice
- Applies iterative reduction (projection & subtraction)
- Preserves the **same lattice** (structure of all integer linear combinations)
- Finds a **shorter and nearly orthogonal** basis

## 📊 Features included:
- Full reduction algorithm (`reduce_2d_basis`)
- Visualizations of the basis before and after reduction
- Randomized testing with lattice-equivalence checks
- Utility functions for table formatting and validation

## 🧠 Why it matters:
Lattice reduction is a fundamental tool in **cryptography**, **geometry of numbers**, and **computational number theory**. This notebook helps you understand how reduction works visually and numerically in 2D before moving to higher dimensions or LLL.

---

### 🔧 Function: `reduce_2d_basis(basis1, basis2)`

This function performs **basis reduction** on two 2D lattice vectors using a simple Gram-Schmidt-like algorithm.

#### 📌 Input:
- `b1`, `b2`: NumPy arrays representing the initial basis vectors (shape `(R^2)` each)

#### ✅ Output:
- A list `[steps, basis1_reduced, basis2_reduced]`, representing the reduced basis vectors, and amount of steps.

In [4]:
import numpy as np

def reduce_2d_basis(basis1, basis2, verbose=False):

    data = []

    steps = 0

    while True:

        if np.linalg.norm(basis2) < np.linalg.norm(basis1):
            basis1, basis2 = basis2, basis1
            continue


        t = round(np.dot(basis1, basis2) / np.dot(basis1, basis1))

        data.append({
            'step': steps,
            'b1': basis1.copy(),
            'b2': basis2.copy(),
        })

        steps += 1


        if t == 0:
            break

        basis2 = basis2 - t * basis1

    shortest = basis1 if np.linalg.norm(basis1) <= np.linalg.norm(basis2) else basis2

    data.append({
        'step': '→ shortest',
        'b1': shortest if np.array_equal(shortest, basis1) else '',
        'b2': shortest if np.array_equal(shortest, basis2) else ''
    })

    return data if verbose else [basis1, basis2]

### 📊 2D Basis Reduction Table

This table shows the step-by-step reduction of a 2D lattice basis:

- **`step`** — the number of the reduction step
- **`b1`, `b2`** — the basis vectors at each step
- The final row labeled **`→ shortest`** indicates the shortest vector found in the final reduced basis

The goal of the reduction is to simplify the basis while preserving the original lattice:
- Making vectors shorter
- Moving them closer to orthogonality

In [5]:
import pandas as pd
from IPython.display import display

def build_basis_table(data):

    rows = []
    for entry in data:
        b1 = entry['b1']
        b2 = entry['b2']

        def format_vec(vec):
            if isinstance(vec, (np.ndarray, list, tuple)) and len(vec) == 2:
                return f"[{vec[0]}, {vec[1]}]"
            elif isinstance(vec, str):
                return vec
            else:
                return ""

        rows.append({
            'step': entry['step'],
            'b1': format_vec(b1),
            'b2': format_vec(b2),
        })

    return pd.DataFrame(rows)


b1 = np.array([31, 59])
b2 = np.array([37, 70])

data = reduce_2d_basis(b1, b2, verbose = True)
table = build_basis_table(data)
display(table.style.hide(axis="index"))

step,b1,b2
0,"[31, 59]","[37, 70]"
1,"[6, 11]","[31, 59]"
2,"[1, 4]","[6, 11]"
3,"[3, -1]","[1, 4]"
→ shortest,"[3, -1]",


# ✅ Lattice Reduction Test Log

**Test configuration:**

- Total tests: `5`
- Vector range: `[-50, 50]`
- Min |component|: `10`
- Algorithm: `2D basis reduction`
- Check:
  - ✅ Same lattice (via invertible integer transformation)
  - ✅ Shorter vector after reduction

In [6]:
def check_same_lattice(basis1_init, basis2_init, basis1_reduced, basis2_reduced):

    Basis_initial = np.column_stack([basis1_init, basis2_init])
    Basis_reduced = np.column_stack([basis1_reduced, basis2_reduced])

    T = np.linalg.solve(Basis_initial, Basis_reduced)

    return np.allclose(T, np.round(T)) and round(abs(np.linalg.det(T))) == 1


def generate_n_2d_bases(n=10, min_abs=10, max_val=50):

    bases = []

    while len(bases) < n:
        b1 = np.random.randint(-max_val, max_val + 1, size=2)
        b2 = np.random.randint(-max_val, max_val + 1, size=2)

        all_coords = np.concatenate([b1, b2])
        if any(abs(x) < min_abs for x in all_coords):
            continue

        if np.linalg.matrix_rank(np.column_stack([b1, b2])) == 2:
            bases.append((b1, b2))

    return bases


def tests_br2d(basis_list, verbose=False):

    tests_amount = len(basis_list)
    tests_passed = 0

    results = []

    for i, (b1, b2) in enumerate(basis_list):
        b1_reduced, b2_reduced = reduce_2d_basis(b1, b2)

        same = check_same_lattice(b1, b2, b1_reduced, b2_reduced)
        original_len = min(np.linalg.norm(b1), np.linalg.norm(b2))
        reduced_len = min(np.linalg.norm(b1_reduced), np.linalg.norm(b2_reduced))
        improved = reduced_len <= original_len

        if same and improved:
            tests_passed += 1
            if verbose:
                print(f"✅ Test {i + 1}: PASSED")
                print(f"Initial basis: b1 = {b1}, b2 = {b2}")
                print(f"Reduced basis: b1 = {b1_reduced}, b2 = {b2_reduced}")

            results.append({
                "b1": b1_reduced,
                "b2": b2_reduced,
                "result": 1
            })

        else:

            if verbose:
                print(f"❌ Test {i + 1} FAILED: ")
                print(f"b1 = {b1}, b2 = {b2}")

            results.append({
                    "b1": b1_reduced,
                    "b2": b2_reduced,
                    "result": 0
            })
    if verbose:
        print(f"\n📊 {tests_amount}/{tests_passed} tests passed.")

    return results

basis_list = generate_n_2d_bases(10)
tests_br2d(basis_list, verbose=True);

✅ Test 1: PASSED
Initial basis: b1 = [ 35 -47], b2 = [ 25 -28]
Reduced basis: b1 = [ -5 -10], b2 = [-15   9]
✅ Test 2: PASSED
Initial basis: b1 = [ 48 -13], b2 = [-46 -12]
Reduced basis: b1 = [  2 -25], b2 = [-46 -12]
✅ Test 3: PASSED
Initial basis: b1 = [-32  44], b2 = [11 39]
Reduced basis: b1 = [11 39], b2 = [-43   5]
✅ Test 4: PASSED
Initial basis: b1 = [ 11 -15], b2 = [ 31 -34]
Reduced basis: b1 = [ 9 -4], b2 = [-7 -7]
✅ Test 5: PASSED
Initial basis: b1 = [ 32 -45], b2 = [ 42 -13]
Reduced basis: b1 = [-10 -32], b2 = [ 42 -13]
✅ Test 6: PASSED
Initial basis: b1 = [ 25 -50], b2 = [-43  23]
Reduced basis: b1 = [-18 -27], b2 = [-43  23]
✅ Test 7: PASSED
Initial basis: b1 = [-48 -10], b2 = [35 33]
Reduced basis: b1 = [-13  23], b2 = [35 33]
✅ Test 8: PASSED
Initial basis: b1 = [ 15 -21], b2 = [35 15]
Reduced basis: b1 = [ 15 -21], b2 = [35 15]
✅ Test 9: PASSED
Initial basis: b1 = [21 41], b2 = [-44  24]
Reduced basis: b1 = [21 41], b2 = [-44  24]
✅ Test 10: PASSED
Initial basis: b1 = [